## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset ##

In [1]:
!ls

abcnews-date-text.csv		   Latent_dirichlet_allocation_solutions.ipynb
Latent_dirichlet_allocation.ipynb


In [2]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv');
# We only need the Headlines text column from the data
data_text = data[['headline_text']];

In [7]:
'''
Add an index column to the dataset and save the dataset as 'documents'
'''
data_text['index'] = data_text.index
documents = data_text
documents.tail()

,headline_text,index
1103660,the ashes smiths warners near miss liven up bo...,1103660
1103661,timelapse: brisbanes new year fireworks,1103661
1103662,what 2017 meant to the kids of australia,1103662
1103663,what the papodopoulos meeting may mean for ausus,1103663
1103664,who is george papadopoulos the former trump ca...,1103664


In [8]:
'''
Get the total number of documents
'''
print(len(documents))

1103665


In [21]:
'''
Preview a document and assign the index value to 'document_num'
'''
document_num = 4311
print("\n**Printing out a sample document:**")
print(documents[documents['index'] == document_num])


**Printing out a sample document:**
                                          headline_text  index
4311  ratepayers group wants compulsory local govt v...   4311


In [22]:
'''
Seperate the value of the headline from the document selected with 'document_num'
'''
documents['headline_text'].iloc[document_num]

'ratepayers group wants compulsory local govt voting'

** Looks like this document will come under `Politics`. **

## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [23]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [24]:
'''
Lemmatizing example for a verb, noun.
'''
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


In [25]:
'''
Stemming example
'''
stemmer = SnowballStemmer("english")
plurals = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in plurals]
print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [26]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result = lemmatize_stemming(text)
    return result

In [27]:
'''
Preview a document after preprocessing
'''

doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


Tokenized and lemmatized document: 
ratepayers group wants compulsory local govt vot


In [30]:
'''
Save the values of the headlines into a variable 'training_headlines' from 'documents'
'''
training_headlines = documents['headline_text'].values

In [31]:
'''
Preview 'training_headlines'
'''
training_headlines

array(['aba decides against community broadcasting licence',
       'act fire witnesses must be aware of defamation',
       'a g calls for infrastructure protection summit', ...,
       'what 2017 meant to the kids of australia',
       'what the papodopoulos meeting may mean for ausus',
       'who is george papadopoulos the former trump campaign aide'], dtype=object)

In [37]:
'''
Perform preprocessing on entire dataset using the function defined earlier and save it to 'processed_docs'
'''
processed_docs = [preprocess(doc) for doc in training_headlines]

In [38]:
'''
Preview 'processed_docs'
'''
processed_docs

['aba decides against community broadcasting lic',
 'act fire witnesses must be aware of defam',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay ris',
 'air nz strike to affect australian travel',
 'ambitious olsson wins triple jump',
 'antic delighted with record breaking barca',
 'aussie qualifier stosur wastes four memphis match',
 'aust addresses un security council over iraq',
 'australia is locked into war timetable opp',
 'australia to contribute 10 million in aid to iraq',
 'barca take record as robson celebrates birthday in',
 'bathhouse plans move ahead',
 'big hopes for launceston cycling championship',
 'big plan to boost paroo water suppli',
 'blizzard buries united states in bil',
 'brigadier dismisses reports troops harassed in',
 'british combat troops arriving daily in kuwait',
 'bryant leads lakers to double overtime win',
 'bushfire victims urged to see centrelink',
 'businesses should prepare for terrorist attack',
 'calleri 

## Step 3.1: Bag of words on the dataset ##

In [46]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears in the training set using gensim.corpora.Dictionary
and call it 'dictionary'
'''
processed_token_docs = [doc.split(' ') for doc in processed_docs]
dictionary = gensim.corpora.Dictionary(processed_token_docs)

In [47]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 aba
1 against
2 broadcasting
3 community
4 decides
5 lic
6 act
7 aware
8 be
9 defam
10 fire


** Gensim filter_extremes **

`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [48]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1)

** Gensim doc2bow **

`doc2bow(document)`

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [49]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_token_docs]

In [50]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
# document_num = 4311
bow_corpus[document_num]

[(206, 1), (307, 1), (777, 1), (1853, 1), (2164, 1), (5391, 1), (5392, 1)]

In [51]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4311 which we have checked in Step 2
bow_doc_4311 = bow_corpus[document_num]

for i in range(len(bow_doc_4311)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4311[i][0], 
                                                     dictionary[bow_doc_4311[i][0]], 
                                                     bow_doc_4311[i][1]))

Word 206 ("govt") appears 1 time.
Word 307 ("group") appears 1 time.
Word 777 ("local") appears 1 time.
Word 1853 ("wants") appears 1 time.
Word 2164 ("vot") appears 1 time.
Word 5391 ("compulsory") appears 1 time.
Word 5392 ("ratepayers") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [57]:
# For very common words like stop words TF-IDF should automatically 'get rid of' these
#by yielding a very small value
TF = 15 / 100 #hypothetical "the" apears sixty times in a single doc
IDF = np.log(10000000 / 9999999) #"the" apears in every doc except for one
TF_IDF = TF * IDF
TF_IDF

1.500000075755899e-08

In [56]:
# For more unique words, like "tiger", TI-IDF yields a larger value
TF = 3 / 100 #hypothetical "tiger" apears six times in a single doc
IDF = np.log(10000000 / 1000) #"tiger" apears in 1000 docs of the 10 million
TF_IDF = TF * IDF
TF_IDF

0.2763102111592855

In [59]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
tfidf = models.TfidfModel(corpus=bow_corpus)

In [68]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
corpus_tfidf = tfidf[bow_corpus]

In [69]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.49846725405078546),
 (1, 0.23281580397526147),
 (2, 0.50816243532606298),
 (3, 0.28180150161547851),
 (4, 0.44985806747916762),
 (5, 0.39662799975931773)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [71]:
# LDA mono-core
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics = 10, 
                                       id2word = dictionary,                                    
                                       passes = 50)

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 289, in worker_e_step
    worker_lda.do_estep(chunk)  # TODO: auto-tune alpha?
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/opt/conda/lib/python3.6/site-packages/gensim/models/ldamodel.py", line 533, in do_estep
    gamma, sstats = self.inference(chunk, collect_sstats=True)
  File "/opt/conda/lib/python3.6/site-packages/gensim/models/ldamodel.py", line 495, in inference
    gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [ ]:
'''
Define lda model using tfidf corpus
'''
lda_model_tfidf = gensim.models.LdaMulticore(# TODO)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [ ]:
'''
Text of sample document 4310
'''
print(train_headlines[4310])

In [ ]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310

# TODO

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [ ]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
document_num = 4310
# Our test document is document number 4310
# TODO

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Step 6: Testing model on unseen document ##

In [ ]:
unseen_document = # TODO

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

# TODO

The model correctly classifies the unseen document with 'x'% probability to the X category.